In [2]:
Z2 = GF(2)

In [3]:
G = matrix(Z2, [[1,1,0,1,0,0,0],
                [0,1,1,0,1,1,0],
                [1,0,0,1,1,0,1]])
G

[1 1 0 1 0 0 0]
[0 1 1 0 1 1 0]
[1 0 0 1 1 0 1]

In [4]:
G.rank()

3

In [5]:
C = LinearCode(G)

In [6]:
C

[7, 3] linear code over GF(2)

In [7]:
d = C.minimum_distance()
d

3

In [8]:
mens_limpa = vector(Z2, [1,1,1,1,0,1,1])
mens_limpa in C

True

In [9]:
mens_err = mens_limpa + vector([0,1,0,0,0,0,0]) + 
mens_err, mens_err in C

((1, 0, 1, 1, 0, 1, 1), False)

In [17]:
H*(matrix(mens_limpa).transpose()) #Hc = 0


[0]
[0]
[0]
[0]

In [20]:
sindrome = H*(matrix(mens_err).transpose()) #Hr != 0
sindrome

[0]
[1]
[0]
[0]

In [21]:
#Correção
I7 = identity_matrix(7)
I7

[1 0 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 0 1 0 0 0 0]
[0 0 0 1 0 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 0 1 0]
[0 0 0 0 0 0 1]

In [38]:
[(H*matrix(col).transpose(), f"Bit {i+1}") for i,col in enumerate(I7.columns())]

[(
[1]         
[0]         
[0]         
[0], 'Bit 1'
),
 (
[0]         
[1]         
[0]         
[0], 'Bit 2'
),
 (
[0]         
[0]         
[1]         
[0], 'Bit 3'
),
 (
[1]         
[1]         
[0]         
[0], 'Bit 4'
),
 (
[0]         
[0]         
[0]         
[1], 'Bit 5'
),
 (
[0]         
[1]         
[1]         
[1], 'Bit 6'
),
 (
[0]         
[1]         
[0]         
[1], 'Bit 7'
)]

**Podemos observar que é segundo bit que está errado como era esperado, visto que o resultado da multiplicação da matriz de paridade H pela segunda coluna da matriz identidade deu o resultado do síndrome (H*mensagem_errada)**

In [39]:
mens_err

(1, 0, 1, 1, 0, 1, 1)

In [42]:
mens_corrigida = mens_err + vector([0,1,0,0,0,0,0])
mens_corrigida, mens_corrigida in C, mens_corrigida == mens_limpa

((1, 1, 1, 1, 0, 1, 1), True, True)